## Existing magics

In [1]:
%%html
<h1>Heading 1</h1>

In [3]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
%%R
x <- 42
print(x)

[1] 42


## New magics

In [5]:
import re
from IPython.display import HTML
from IPython.core.magic import register_cell_magic

In [6]:
def var_sub(m):
  v = globals()[m[1]]
  if type(v) != str:
    v = repr(v)
  return v

In [7]:
@register_cell_magic
def javascript(line="", cell=""):
  # var sub
  cell = re.sub(r'\{\{(\w+)+}}', var_sub, cell)

  m = re.match(r'\w+', line)
  if m:                       # assign to a variable
    globals()[m[0]] = cell
  else:                       # normal display
    display(HTML("""
<div id="drawing"></div>
<script src="https://cdnjs.cloudflare.com/ajax/libs/svg.js/2.6.5/svg.js"></script>
<script>
var draw = SVG('drawing').size('100%', 100)
var y_cursor_pos = 0
function print(x){
  draw.text(x.toString()).move(400, y_cursor_pos)
  y_cursor_pos += 20
}

text = s=> draw.text(s).font('anchor', 'middle')
line = (x1,y1, x2,y2)=> draw.line(x1,y1, x2, y2).stroke({width: 1})
max = Math.max
""" + cell))

In [8]:
%%javascript

draw.rect(100, 100)

## Elasticsearch magic

In [13]:
# install
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.10.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.10.0

In [14]:
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.10.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [15]:
# wait a bit then test
!curl -X GET "localhost:9200/"

{
  "name" : "45f0c03f1a57",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "DIyC9u2gSBOahBVnVu1cjA",
  "version" : {
    "number" : "7.10.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "51e9d6f22758d0374a0f3f5c6e8f3a7997850f96",
    "build_date" : "2020-11-09T21:30:33.964949Z",
    "build_snapshot" : false,
    "lucene_version" : "8.7.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [16]:
from requests import models, Session, Request
from urllib.parse import urljoin
from IPython.core.magic import register_cell_magic
import re

# to display json response
def render(r):
  text = r.text
  if text[0] in "[{":  # really json
    return """
    <script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
    <script>
    renderjson.set_show_to_level(1)
    document.body.appendChild(renderjson(%s))
    new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
    </script>
    """ % text
  else:    # other status text
    return "<pre>%s</pre>" % text
models.Response._repr_html_ = render

In [17]:
# %%es magic
@register_cell_magic
def es(line=None, cell=""):
  cell = re.sub(r'(?m)^\s*#.*\n?','', cell) # remove comment
  line1 = (cell + '\n').find('\n')
  method, path = cell[:line1].split(None, 1)
  body = cell[line1:].strip()
  args = {}
  if body:
    args['data'] = (body + '\n').encode()  # in case _bulk
    args['headers'] = {'Content-Type': 'application/json'}

  rsp = Session().send(
          Request(method, urljoin('http://localhost:9200', path), **args)
            .prepare())
  return rsp

In [18]:
%%es
GET /

<Response [200]>

In [19]:
%%es
PUT /customer/_doc/1
{
  "name": "Harry Potter"
}

<Response [201]>

In [20]:
%%es
PUT customer/_doc/2
{
  "name": "Draco Malfoy"
}

<Response [201]>

## Python calls JS

In [21]:
%%html
<script>
document.body.append(document.createTextNode("Hello SJSU!"))
</script>

In [22]:
from IPython.display import Javascript
Javascript('document.body.append(document.createTextNode("Hello SJSU!"))')

<IPython.core.display.Javascript object>

In [24]:
from google.colab.output import eval_js
eval_js('2+4')

6

## JS calls Python

In [25]:
from IPython.display import JSON
import IPython.display as idisplay
from google.colab import output

def concat(a, b):
  return JSON({'result': '%s %s' % (a, b)})

output.register_callback('notebook.concat', concat)

In [26]:
idisplay.Javascript('''
(async function() {

  const result = await google.colab.kernel.invokeFunction(
    'notebook.concat',
    ['hello', 'world!'],
    {});

  const text = result.data['application/json'].result;

  document.querySelector("#output-area").appendChild(document.createTextNode(text))

})();
''')

<IPython.core.display.Javascript object>

## Take a photo

In [31]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

In [32]:
VIDEO_HTML = """
<video autoplay
 width=800 height=600></video>
<script>
var video = document.querySelector('video')
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""

In [33]:
def take_photo(filename='photo.jpg', quality=0.8):
  display(HTML(VIDEO_HTML % quality))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)

In [34]:
take_photo()

27373